In [1]:
import os
import os.path as osp
import pickle
import csv
import collections

import numpy as np
import PIL.Image as Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.transforms import Compose, Normalize, Resize
import pandas as pd

In [2]:
# Reading the csv file given for the mini-imagenet dataset
dataset = pd.read_csv('datasets/data/mini-imagenet/train.csv', low_memory=False)
dataset

,filename,label
0,n0153282900000005.jpg,n01532829
1,n0153282900000006.jpg,n01532829
2,n0153282900000007.jpg,n01532829
3,n0153282900000010.jpg,n01532829
4,n0153282900000014.jpg,n01532829
...,...,...
38395,n1313361300001288.jpg,n13133613
38396,n1313361300001290.jpg,n13133613
38397,n1313361300001296.jpg,n13133613
38398,n1313361300001297.jpg,n13133613


In [4]:
# Unique classes
classes=dataset.label.unique()
classes.shape

(64,)

In [14]:
# Create 2 classes list to form a group of 5 classes and 10 classes respectively
classes_5=classes[:5]
classes_10=classes[:10]
print(classes_5,classes_10)

['n01930112' 'n01981276' 'n02099601' 'n02110063' 'n02110341'] ['n01930112' 'n01981276' 'n02099601' 'n02110063' 'n02110341' 'n02116738'
 'n02129165' 'n02219486' 'n02443484' 'n02871525']


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Define transforms for data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(64, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load MiniImageNet dataset
trainset = torchvision.datasets.CIFAR100(root='datasets', train=True,
                                             download=True,
                                             transform=transform_train)

# Define dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

# Define ResNet-50 model
class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        self.resnet = torchvision.models.resnet50(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 64)

    def forward(self, x):
        x = self.resnet(x)
        return x

model = ResNet50()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting datasets\cifar-100-python.tar.gz to datasets


c:\Users\kanad\Desktop\Github repos\IE_506_PROJECT\ML_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\kanad\Desktop\Github repos\IE_506_PROJECT\ML_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\kanad\Desktop\Github repos\IE_506_PROJECT\ML_env\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\Users\kanad\Desktop\Github repos\IE_506_PROJECT\ML_env\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\kanad\Desktop\Github repos\IE_506_PROJECT\ML_env\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\kanad\Desktop\Github repos\IE_506_PROJECT\ML_env\lib\site-packages\torchvision\datasets\cifar.py", line 118, in __getitem__
    img = self.transform(img)
  File "c:\Users\kanad\Desktop\Github repos\IE_506_PROJECT\ML_env\lib\site-packages\torchvision\transforms\transforms.py", line 94, in _